<a href="https://colab.research.google.com/github/lacykaltgr/continual-learning-ait/blob/experiment/classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
'''Download the files '''
'''Only for colab'''

!wget https://github.com/lacykaltgr/continual-learning-ait/archive/refs/heads/main.zip
!unzip main.zip
!find continual-learning-ait-main -type f ! -name "main.ipynb" -exec cp {} . \;

!rm -r stable_diffusion
!mkdir stable_diffusion
!mv diffusion_model.py stable_diffusion/
!mv autoencoder_kl.py stable_diffusion/
!mv layers.py stable_diffusion/
!mv stable_diffusion.py stable_diffusion/
!mv constants.py stable_diffusion/

--2023-05-09 02:24:21--  https://github.com/lacykaltgr/continual-learning-ait/archive/refs/heads/main.zip
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/lacykaltgr/continual-learning-ait/zip/refs/heads/main [following]
--2023-05-09 02:24:21--  https://codeload.github.com/lacykaltgr/continual-learning-ait/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 20.27.177.114
Connecting to codeload.github.com (codeload.github.com)|20.27.177.114|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip.1’

main.zip.1              [ <=>                ] 338.93K  1.77MB/s    in 0.2s    

2023-05-09 02:24:22 (1.77 MB/s) - ‘main.zip.1’ saved [347065]

Archive:  main.zip
4e98f3b1d72a496ef351c2ffa7a12a2d9dddf2d8
replace continual-learning-ait-main/REA

In [3]:
import tensorflow as tf
import keras
from stable_diffusion.layers import PaddedConv2D
from stable_diffusion.autoencoder_kl import ResnetBlock, AttentionBlock
from classifier import GatedDense

In [4]:
def load_cifar_10():
    (X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
    n_classes = 10
    X_train = (X_train / 127.5) -1
    X_test = (X_test / 127.5) -1
    y_train = tf.keras.utils.to_categorical(y_train, n_classes)
    y_test = tf.keras.utils.to_categorical(y_test, n_classes)
    return (X_train, y_train), (X_test, y_test)

In [19]:
class Classifier(keras.Sequential):
    def __init__(self):
        print("Encoder init")
        super().__init__(
            [
                PaddedConv2D(64, 3, padding=1 ),
                ResnetBlock(64,64),
                ResnetBlock(64, 64),
                PaddedConv2D(64 , 3 , padding=(0,1), stride=2),

                ResnetBlock(64,128),
                AttentionBlock(128), 
                ResnetBlock(128, 128),
                PaddedConv2D(128 , 3 , padding=(0,1), stride=2),

                ResnetBlock(128, 256),
                ResnetBlock(256, 256),
                PaddedConv2D(256, 3, padding=(0,1), stride=2),

                #ResnetBlock(512,512),
                #ResnetBlock(512, 512),

                #ResnetBlock(512, 512),
                
                #ResnetBlock(512, 512),

                keras.layers.GroupNormalization(epsilon=1e-5),
                keras.layers.Activation("swish"),
                PaddedConv2D(8, 3, padding=1 ),
                PaddedConv2D(8, 1 ),
                keras.layers.Lambda(lambda x : x[..., :4] * 0.18215),

                #############################

                keras.layers.Flatten(),
                GatedDense(32, activation=keras.activations.relu),
                keras.layers.Dropout(0.2),
                GatedDense(32, activation=None),
                keras.layers.Dense(10, activation='softmax')
            ]
        )

In [20]:
(X_train, y_train), (X_test, y_test) = load_cifar_10()

model = Classifier()
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-2),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=[keras.metrics.CategoricalAccuracy()]
)
model.fit(X_train, y_train, batch_size=128, epochs=100, validation_data=(X_test, y_test))

Encoder init
Epoch 1/100
391/391 [==============================] - 80s 187ms/step - loss: 1.8708 - categorical_accuracy: 0.3112 - val_loss: 1.5901 - val_categorical_accuracy: 0.4252
Epoch 2/100
391/391 [==============================] - 72s 184ms/step - loss: 1.5588 - categorical_accuracy: 0.4313 - val_loss: 1.3925 - val_categorical_accuracy: 0.4915
Epoch 3/100
164/391 [===========>..................] - ETA: 38s - loss: 1.4352 - categorical_accuracy: 0.4757

KeyboardInterrupt: ignored